In [32]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [33]:
with open('../Corpora/movies-pp.txt', 'r') as file:
    # Movies is a list where each line is a document
    movies = file.readlines()

# TODO: Represent the corpus as a Class
# A list with every word in the corpus
flat_list = [word for movie in movies for word in movie.split()]

# Set with unique tokens
vocab = set(flat_list)

# Size of vocabulary
W = len(vocab)

## TODO: initialize docs_list as a list compreehension
# docs_list is a list containing the document indexes on the 'movies' list for each word
docs_list = []
step = 0
for movie in movies:
    for word in movie.split():
        docs_list.append(step)
    step+=1
##

# doc_len
doc_lens = []
for movie in movies:
    size = len(movie.split())
    for word in movie.split():
        doc_lens.append(size)
    

## Initial Parameters

In [34]:
alpha = 0.02
beta = 0.1
iter_n = 500
topic_n = 20

# Initializing Z

Contains the topic assignment for each word in the corpus.

In [35]:
# Initializing Z as a pandas DataFrame ()

z = pd.DataFrame(data={'words':flat_list,
                       'topic':np.random.randint(0, topic_n, len(flat_list)),
                       'doc':docs_list,
                      'doc_size':doc_lens})

print(z.head())

     words  topic  doc  doc_size
0     2000     14    0         1
1     plot     14    1       332
2      two     18    1       332
3     teen     11    1       332
4  couples      4    1       332


# Initializing Nd,k

contains the number of words assigned to each topic in a document

In [36]:
# Creating a dataframe with counts of words assigned to each topic
N_d_k = pd.DataFrame()

# Iterating over the documents in the 'movies' list of documents
for i in tqdm(range(len(movies))):
    # Series with zeroes initialized with length topic_n
    init_row = pd.Series(np.zeros(topic_n), dtype='int64')
    
    # Creating a dataframe with words within this document
    doc = z[z['doc']==i]
    
    # summing the zeros row with the count of words assigned to each topic.
    # performs sum on index
    w_counts = doc['topic'].value_counts() + init_row
    
    # Filling NaN values to zero
    w_counts = w_counts.fillna(0)
    
    # Appends 
    N_d_k = N_d_k.append(w_counts, ignore_index=True)


In [37]:
print(len(N_d_k))
N_d_k.head()

2001


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,20.0,24.0,16.0,15.0,15.0,11.0,11.0,18.0,18.0,14.0,15.0,17.0,21.0,11.0,23.0,18.0,12.0,13.0,23.0,17.0
2,10.0,2.0,4.0,9.0,4.0,6.0,3.0,7.0,4.0,7.0,5.0,7.0,7.0,8.0,5.0,13.0,6.0,8.0,5.0,9.0
3,11.0,23.0,16.0,11.0,9.0,20.0,13.0,13.0,11.0,17.0,15.0,10.0,16.0,9.0,16.0,13.0,8.0,7.0,15.0,10.0
4,17.0,11.0,13.0,14.0,10.0,12.0,11.0,5.0,12.0,13.0,13.0,19.0,10.0,12.0,14.0,17.0,14.0,17.0,16.0,14.0


## Initializing Nk,w

the number of times the word W is assigned to the topic K

In [38]:
N_k_w = pd.DataFrame(index = set(flat_list))

for topic in range(topic_n):
    doc = z[z['topic']==topic]
    doc = doc['words'].value_counts()
    N_k_w[topic] = doc
    
N_k_w = N_k_w.fillna(0)

In [39]:
N_k_w.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
mesmerizing,1.0,4.0,4.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,3.0,1.0
reruns,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chow,0.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,3.0,1.0,4.0,1.0,3.0,3.0,2.0,0.0,1.0,0.0,2.0,1.0
asp,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jeb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
N_k_w[1]['chow']

2.0

## Initializing nk

The number of times any word is assigned to topic k

In [40]:
N_k = N_k_w.sum()

In [49]:
print(N_k.head())

0    35019.0
1    34597.0
2    34850.0
3    35070.0
4    34979.0
dtype: float64


## Initializing Nw_d



In [43]:
Nw_d = z[['doc','doc_size']].set_index('doc').drop_duplicates()
Nw_d.head()

,doc_size
doc,
0,1
1,332
2,129
3,263
4,264


# Actuall LDA 

* Nk_w: the number of times the **current word** appears with the **current topic**
* N_k: The number of times the **current topic** appears in total
* N_d_k: The number of times the **current topic** appears in the **current document**
* Nw_d: The number of **tokens** in the **current document**

In [ ]:
for i in range(iter_n):
    for index in range(len(z)):
        word = z['words'][index]
        topic = z['topic'][index]
        doc = z['doc'][index]
        
        
        # TODO: subtract one (-=) from the correct tables
        # Access the rows on the tables 
        Nwk -= Nk_w[topic][word]
        -= N_k
        
        for i in range(topic_n):
            p = ((Nwk + beta)/(N_k + beta*W)) * ((N_d_k + alfa)/(Nw_d + topic_n*alfa)
            

# Pseudo-Code

```Input: words w ∈ documents d  
Output: topic assignments z and counts nd,k , nk,w , and nk  
begin    
    randomly initialize z and increment counters  
    foreach iteration do  
        for i = 0 → N − 1 do  
            word ← w[i]  
            topic ← z[i]  
            nd,topic -=1; nword,topic -=1; ntopic -=1  
            for k = 0 → K − 1 do  
                p(z = k|·) = (nd,k + αk ) nkk,w  +βw+β×W  
            end  
            topic ← sample from p(z|·)  
            z[i] ← topic  
            nd,topic +=1; nword,topic +=1; ntopic +=1  
        end    
    end  
    return z, nd,k , nk,w , nk  
end  ```